In [7]:
from datetime import datetime, timedelta
import time
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region = boto3.Session().region_name
#role = get_execution_role()
role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"
sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the model artifacts will be uploaded
bucket = sm_session.default_bucket()
prefix = "sagemaker/DEMO-VariantTargeting"

print(region)

[01/31/25 22:13:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=705779;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=665902;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

[01/31/25 22:13:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=400068;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=661428;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=214816;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=228229;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

us-west-1


In [10]:
from sagemaker.session import production_variant
from sagemaker.amazon.amazon_estimator import get_image_uri

model_url1 = f"s3://sagemaker-us-west-1-798223350085/ml_deploy/output/regression-369a577b6d40410b9052679c2d0ba943/output/model.tar.gz"
model_url2 = f"s3://sagemaker-us-west-1-798223350085/ml_deploy/output/regression-ba65a86c01c8407ba75b475abdc045ec/output/model.tar.gz"

model_name1 = f"DEMO-xgb-churn-pred1-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name2 = f"DEMO-xgb-churn-pred2-{datetime.now():%Y-%m-%d-%H-%M-%S}"
image_uri1 = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')
image_uri2 = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-2')

sm_session.create_model(name=model_name1, role=role, container_defs={
    'Image': image_uri1,
    'ModelDataUrl': model_url1
})

sm_session.create_model(name=model_name2, role=role, container_defs={
    'Image': image_uri2,
    'ModelDataUrl': model_url2
})


[01/31/25 22:15:33] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=161014;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=501551;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Defaulting to only available Python version: py3                     ]8;id=114277;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=718121;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py#604\604]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=90184;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=246405;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=170403;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=531482;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Defaulting to only available Python version: py3                     ]8;id=909155;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=971284;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py#604\604]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=267234;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=410522;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     Creating model with name: DEMO-xgb-churn-pred1-2025-01-31-22-15-33     ]8;id=923723;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=870040;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/31/25 22:15:34] INFO     Creating model with name: DEMO-xgb-churn-pred2-2025-01-31-22-15-33     ]8;id=697955;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=20434;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

'DEMO-xgb-churn-pred2-2025-01-31-22-15-33'

In [11]:
variant1 = production_variant(
    model_name=model_name1,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant1",
    initial_weight=1,
)
variant2 = production_variant(
    model_name=model_name2,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant2",
    initial_weight=1,
)

(variant1, variant2)

({'VariantName': 'Variant1',
  'ModelName': 'DEMO-xgb-churn-pred1-2025-01-31-22-15-33',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.4xlarge'},
 {'VariantName': 'Variant2',
  'ModelName': 'DEMO-xgb-churn-pred2-2025-01-31-22-15-33',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.4xlarge'})

In [12]:
endpoint_name = f"xgb-churn-pred-abtest"
print(f"EndpointName={endpoint_name}")
sm_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)


EndpointName=xgb-churn-pred-abtest


[01/31/25 22:17:14] INFO     Creating endpoint-config with name xgb-churn-pred-abtest               ]8;id=417327;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=550503;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name xgb-churn-pred-abtest                      ]8;id=314449;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=112549;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-----!

'xgb-churn-pred-abtest'

In [15]:
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    TargetVariant='Variant2',
    ContentType='text/csv',
    Body = "97.0,0.0,3.122199071584636,5.0,3.293510068877556,10.0,4.6638374904590005,150.0,5.1467102459546235,5.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0"
)
print(response['Body'].read().decode('utf-8'))

0.23035253584384918


In [16]:
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    TargetVariant='Variant1',
    ContentType='text/csv',
    Body = "97.0,0.0,3.122199071584636,5.0,3.293510068877556,10.0,4.6638374904590005,150.0,5.1467102459546235,5.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0"
)
print(response['Body'].read().decode('utf-8'))

0.084608294069767
